In [1]:
!pip install selenium-wire selenium webdriver-manager fake_useragent undetected-chromedriver --upgrade undetected-chromedriver

In [13]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException, TimeoutException, ElementNotInteractableException
from bs4 import BeautifulSoup
import time
import pandas as pd

In [14]:
from urllib.parse import urlparse, urljoin
url = 'https://www.tripadvisor.com/Restaurant_Review-g47958-d10624330-Reviews-MIX_Kitchen_and_Bar-Ithaca_Finger_Lakes_New_York.html'

In [15]:
from fake_useragent import UserAgent
import time
import random 

ua = UserAgent()
headers = {"User-Agent": ua.random}
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"


using user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36


In [16]:
import undetected_chromedriver as uc
def get_driver():
    """Initialize and configure Chrome WebDriver"""
    options = uc.ChromeOptions()
    options.add_argument(f'user-agent={headers}')
    options.add_argument('--lang=en-US')
    options.add_argument('--disable-notifications')
    
    driver = uc.Chrome(headless=False, options=options)
    driver = webdriver.Chrome(options=options)
    
    driver.maximize_window()
    driver.execute_cdp_cmd(
        "Page.addScriptToEvaluateOnNewDocument",
        {"source": "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"}
    )
    return driver

In [17]:
def accept_cookies(driver):
    """Handle cookie acceptance popup"""
    try:
        cookie_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))
        )
        cookie_btn.click()
        print("Cookies accepted")
    except Exception as e:
        print(f"Cookie acceptance failed: {str(e)}")

In [18]:
import random
import time

def scroll_until_all_reviews_loaded(driver, pause_range=(0.5, 1.2), max_attempts=20):
    last_count = 0
    for attempt in range(max_attempts):
        review_blocks = driver.find_elements(By.XPATH, "//div[contains(@class, 'YHQtA')]")
        current_count = len(review_blocks)
        print(f"Scroll attempt {attempt+1}: {current_count} reviews")

        scroll_step = random.randint(180, 360)
        if random.random() < 0.1:
            driver.execute_script("window.scrollBy(0, 300);")
        pause = random.uniform(*pause_range)
        if random.random() < 0.15:
            pause += random.uniform(0.5, 1.2)
        time.sleep(pause)

        if current_count == last_count:
            print("Finished scrolling this page.")
            break
        last_count = current_count

    return driver.find_elements(By.XPATH, "//div[contains(@class, 'YHQtA')]")

In [19]:
def click_next_page(driver):
    try:
        next_button = driver.find_element(By.XPATH, "//a[@aria-label='Next page']")
        next_button.click()
        print("Moved to next page...")
        time.sleep(random.uniform(2, 4))  # Let page load
        return True
    except:
        print("No next page button found.")
        return False

In [20]:
def extract_review_info(review):
    try:
        name = review.find_element(By.XPATH, ".//span[contains(@class, 'biGQs')]/a").text.strip()
    except NoSuchElementException:
        name = ''
    #try:
        #date_raw = review.find_element(By.CLASS_NAME, "aVuQn").text
        #date = date_raw.split("•")[0].strip()
    #except NoSuchElementException:
        #date = ''
    try:
        raw_date = review.find_element(By.XPATH, ".//div[contains(@class, 'biGQs') and contains(text(), 'Written')]").text
        date = raw_date.replace("Written", "").strip()
    except:
        date = ''
    try:
        rating_svg = review.find_element(By.CLASS_NAME, "evwcZ")
        rating_text = rating_svg.find_element(By.TAG_NAME, "title").get_attribute("textContent")
        rating = int(rating_text.split(" ")[0]) 
    except:
        rating = None
    try:
        review_text = review.find_element(By.CLASS_NAME, "JguWG").text.strip()
    except:
        review_text = ''
    print(f"Name: {name} | Date: {date} | Rating: {rating}")
    return {
        'Reviewer Name': name,
        'Date': date,
        'Review Text': review_text,
        'Rating out of 5': rating
    }


In [21]:
def scrape_reviews(url, max_pages = 2):
    driver = get_driver()
    driver.get(url)
    time.sleep(30)
    accept_cookies(driver)
    time.sleep(random.uniform(7, 14))

    all_reviews = []

    for i in range(max_pages):
        
        #scroll_until_all_reviews_loaded(driver, pause_range=(0.5, 1.2), max_attempts=20)

        time.sleep(3)
        #review_sections = driver.find_elements(By.XPATH, "//div[contains(@class, 'YHQtA')]")
        review_sections = scroll_until_all_reviews_loaded(driver)
        print(f"Found {len(review_sections)} review blocks on this page.")
        time.sleep(random.uniform(2, 5)) 

        for review in review_sections:
            print(f"\nScraping review {i + 1}")
            data = extract_review_info(review)
            all_reviews.append(data)
        
        if not click_next_page(driver):
            break
    driver.quit()
    return all_reviews


In [23]:
import csv
def save_to_csv(review_data, filename="tripadvisor_reviews.csv"):
    keys = review_data[0].keys()
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(review_data)
    print(f"Saved {len(review_data)} reviews to {filename}")

review_data = scrape_reviews(url, max_pages=7)
save_to_csv(review_data, filename="tripadvisor_reviews.csv")

Cookie acceptance failed: Message: 
Stacktrace:
0   chromedriver                        0x000000010f3c1568 chromedriver + 6088040
1   chromedriver                        0x000000010f3b917a chromedriver + 6054266
2   chromedriver                        0x000000010ee58540 chromedriver + 415040
3   chromedriver                        0x000000010eeaa0a0 chromedriver + 749728
4   chromedriver                        0x000000010eeaa2f1 chromedriver + 750321
5   chromedriver                        0x000000010eefa764 chromedriver + 1079140
6   chromedriver                        0x000000010eed041d chromedriver + 906269
7   chromedriver                        0x000000010eef7a19 chromedriver + 1067545
8   chromedriver                        0x000000010eed01c3 chromedriver + 905667
9   chromedriver                        0x000000010ee9c05a chromedriver + 692314
10  chromedriver                        0x000000010ee9d1b1 chromedriver + 696753
11  chromedriver                        0x000000010f380c9

In [24]:
df = pd.read_csv('tripadvisor_reviews.csv')
df.head()

,Reviewer Name,Date,Review Text,Rating out of 5
0,Abby N,"December 27, 2024","Loved the food, so did my family, but two star...",2
1,Mungus N,"October 18, 2023",What a fantastic place! My family and friends ...,5
2,Mark W,"May 29, 2023",Over the ten years that my two older grandsons...,1
3,Vladimir S,"July 8, 2022",They are proud about their steaks. We went for...,5
4,orrdin,"October 29, 2021",Excellent menu with a huge number of options a...,5
